# Descargando los datos
Para todos los problemas de nuestro mundo es mejor tener datos y basar nuestras decisiones en ellos. Conseguir los datos es lo dificil!

Hemos dicho que queremos aprender sobre la popularidad de tweets - entonces tiene sentido tirarnos directamente hacia este lado y investigar sobre datos de tweets populares!

## Tarea: Como podemos aportar valor al mundo a traves de los datos?

1. Como podemos usar datos para conseguir valor real en el mundo?
2. Que otras piezas del puzle hay en la "cadena de valor"?
3. Que implica esto sobre nuestra metodologia?

* Apoyo a decisiones - mejores decisiones, entender el contexto, experimentar, tener mas seguridad, automatizar (decisiones y flujos)...
* Los otros stakeholders y como impactamos sobre nuestro "cliente final"
* Iteracion rapida...

## Tarea: investigar Twitter:

1. Que cosas podrian ser utiles considerar? Que tipo de cosas podemos querer en nuestra base de datos?
2. Como podemos recoger estos datos?

### Punto 1

* Generalmente es util pensar en terminos de "objetos" de datos - cada objeto siendo algo que informa sobre el problema que estamos tratando (tweets, usuarios, hashtags, photos...)
* Luego, fuera de las carecteristicas basicas, es muy importante recoger información temporal!
* Tenemos problemas con la temporalidad, porque igual seria interesante saber información de los tweets cada dia desde que fue publicado, por ejemplo...

### Punto 2

* Entra dinamicas interesantes aqui - estamos tratando de datos "externos" que siempre es mas dificil que datos "internos" ya que controlamos los internos
* Con datos externos es muy comun tener que explotar scraping de webs o de APIs - si es por API normalmente es más facil
* Otro punto importante es exactamente como creamos nuestro proceso de scraping ya que influye sobre los datos que recogemos
* El problema tipico con los APIs es que estamos limitado por el propio API (rate limiting, formatos, información expuesta...)

## Como lo vamos a hacer?
Ya he hecho esta tarea antes de la clase porque la descarga tarda lo suyo. He optado por el siguente patron:

* Recoger unas cuentas con muchos seguidores (donde encontraremos muchos tweets populares)
* Scrapear la información de estas cuentas a una fecha
* X dias despues, recoger todos sus tweets entre la fecha de descarga de las cuentas y la fecha de hoy

Lo hacemos asi para poder tener el número de seguidores etc. ANTES de crear los tweets.

In [ ]:
from datetime import datetime
import time

import pandas as pd
import twint
import nest_asyncio
nest_asyncio.apply()


users_downloaded_at = '2021-08-11'
download_tweets_from = '2021-08-12'
download_tweets_to = '2021-08-19'

pd.options.display.max_columns = 500

## Descargando los usuarios

In [ ]:
handles = [
    'BarackObama', 'MichelleObama', 'Oprah', 'realmadrid', 'fcbarcelona', 'sanchezcastejon', 'theonion'
]

In [ ]:
results = []
unfound_accounts = []
not_worked = []
for i, username in enumerate(handles):
    if i % 100 == 0:
        print(f'Done with {i}')
    
    c = twint.Config()
    c.Username = username
    c.Pandas = True
    c.Hide_output = True
    
    try:
        twint.run.Lookup(c)
    
        results.append(twint.storage.panda.User_df)
    except ValueError as e:
        if 'Cannot find twitter account' in str(e):
            unfound_accounts.append(username)
        else:
            raise
    except twint.token.RefreshTokenException:
        not_worked.append(username)
        time.sleep(60)
        
        try:
            twint.run.Lookup(c)
            results.append(twint.storage.panda.User_df)
        except:
            print('Getting refresh errors!')

results = pd.concat(results)
results.to_csv(f'{datetime.today().strftime("%Y-%m-%d")}-handles-data.csv', index=False)

print(unfound_accounts)
print(not_worked)

## Descargando los Tweets

In [ ]:
user_details = pd.read_csv(f'{users_downloaded_at}-handles-data.csv').drop_duplicates('id')

In [ ]:
results = []
not_worked = []
unfound_accounts = []
for i, username in enumerate(user_details.username.dropna().unique()):
    if i % 100 == 0:
        print(f'Done with {i}')
    
    c = twint.Config()
    c.Username = username
    c.Since = download_tweets_from
    c.Until = download_tweets_to
    c.Pandas = True
    c.Hide_output = True
    
    try:
        twint.run.Profile(c)

        results.append(twint.storage.panda.Tweets_df)
    except ValueError as e:
        if 'Cannot find twitter account' in str(e):
            unfound_accounts.append(username)
        else:
            raise
    except twint.token.RefreshTokenException:
        not_worked.append(username)
        time.sleep(60)

        try:
            twint.run.Profile(c)
            results.append(twint.storage.panda.Tweets_df)
        except:
            print('Getting refresh errors!')
        
    
results = pd.concat(results)
results.to_csv(f'{users_downloaded_at}-{download_tweets_from}-{download_tweets_to}-tweets-data.csv', index=False)

Cosas a mencionar (por si me olvido!):

* Que cosas nos devuelve Twitter
* La libreria de twint
* Rate limiting y retries
* Guardando lo que vas sacando
* Scripts vs. notebooks

## Creando nuestro data set
Ahora tenemos unos datos sobre tweets - pero como los utilizamos para entender el problema? Analizarlos por supuesto!

Cualquier persona que ha hecho hasta el analisis de datos mas basico en Excel sabe que es muy conveniente tener nuestros datos en tablas. Ya tenemos esto simplemente por como construimos el proceso inicial, pero tenemos los datos en 2 piezas! Esto dificulta ciertos analisis (pero es necesario para otros!) asi que tenemos que ver como resolver este punto.

Primero, echamos un vistazo breve de los datos para entender que cosas hemos podido sacar.

In [ ]:
user_details = pd.read_csv(f'2021-08-11-handles-data.csv').drop_duplicates('id')

In [ ]:
user_details.head()

In [ ]:
tweet_data = pd.read_csv('2021-08-11-2021-08-12-2021-08-19-tweets-data.csv')

In [ ]:
tweet_data.head()

### Tarea: Como podemos sacar valor de estos datos?
Hemos visto, más o menos que informacion somos capaces de sacar de Twitter - pero,

1. Como nos ayuda en resolver nuestro problema?
2. Siempre hablamos de "big data"? Tenemos aqui "big data"? Hace falta? Que perdemos?

### Juntando los datos
Ahora, queremos también juntar los datos para unos analisis que vamos a hacer en las semanas que vienen. Juntar datos es un concepto muy importante. La raiz de ML es permitir a una maquina aprender sobre un comportamiento - teniendo datos que reflejan este comportamiento. Parte de esto es tener nuestros diferentes "fuentes" de informacion sobre los comportamientos de interes - cada columna de cada objeto de datos aporta **algo** sobre nuestro comportamiento.

Por esto, es muy importante poder "juntar" diferentes objetos y fuentes, para tener una foto más completo.

#### Tarea: Como podemos juntar nuestros datos de tweets y los usuarios?

* Follow up: Implica esto algo sobre la granularidad de los datos? Que pasa si nos interesa más entender los usuarios populares en vez de tweets populares?

In [ ]:
# We're doing it live!